# Bayesian Changepoint Detection with multivariate data in Python

This code computes the probability of changepoints (including changes in correlation) in a time series. In this notebook I show how you can use it. This example is modified from Xiang Xuan's thesis Section 3.2.

First let's generate some data and load some modules:

In [2]:
from __future__ import division
import matplotlib.pyplot as plt
import bayesian_changepoint_detection.generate_data as gd
import seaborn
import pandas as pd
import numpy as np
from functools import partial
import matplotlib.cm as cm
import numpy as np
import json
import os
import glob
from os.path import basename,join,dirname
from datetime import datetime
import numpy as np
from scipy.stats import multivariate_normal, norm
from tqdm import tqdm
from bayesian_changepoint_detection.priors import const_prior
from bayesian_changepoint_detection.offline_likelihoods import IndepentFeaturesLikelihood
import bayesian_changepoint_detection.online_likelihoods as online_ll
from bayesian_changepoint_detection.bayesian_models import offline_changepoint_detection 
from bayesian_changepoint_detection.bayesian_models import online_changepoint_detection
from functools import partial
from bayesian_changepoint_detection.hazard_functions import constant_hazard

%matplotlib inline
%load_ext autoreload
%autoreload 2

def find_anomalies(data, threshold=0.01):
    anomalies = []
    for i in range(1, len(data)):
        if data[i] > threshold:
            anomalies.append(i)

    # re-try if threshold doesn't work
    if len(anomalies) == 0:
        head = 5
        data = data[head:]
        anomalies = [np.argmax(data) + head]

    # merge continuous anomalies if the distance are shorter than 5 steps
    merged_anomalies = [] if len(anomalies) == 0 else [anomalies[0]]
    for i in range(1, len(anomalies)):
        if anomalies[i] - anomalies[i-1] > 5:
            merged_anomalies.append(anomalies[i])
    
    return merged_anomalies, anomalies

Use scipy logsumexp().


In [7]:
sparsity = 5 
epsilon = 1e-7

for data_path in tqdm(glob.glob("../cfm/data/fse-ss/**/simple_data.csv", recursive=True)):
    service_metric = basename(dirname(dirname(data_path)))
    case_idx = basename(dirname(data_path))
    
    # if exist, ignore
    if os.path.exists(f"./fse-ss/{service_metric}_{case_idx}.json"):
        continue
        
    
    data = pd.read_csv(data_path)   
    selected_cols = [c for c in data.columns if "latency-50" in c and 'queue-master' not in c]
    data = data[selected_cols]
    data = data.fillna(method="ffill")
    data = data.fillna(0)

    for c in data.columns:
        data[c] = (data[c] - np.min(data[c])) / (np.max(data[c]) - np.min(data[c]))
    data = data.fillna(method="ffill")
    data = data.fillna(0)
    
    data = data.to_numpy()

    # print(data)
    # run 
    start = datetime.now()
    R, maxes = online_changepoint_detection(
            data,
            partial(constant_hazard, 50),
            online_ll.MultivariateT(dims=data.shape[1])
    )
    time_taken = datetime.now() - start
    with open(f"./fse-ss/{service_metric}_{case_idx}.txt", "w") as f:
        f.write(f"Time taken: {time_taken}")

    #plot 

    density_matrix = -np.log(R[0:-1:sparsity, 0:-1:sparsity]+epsilon)


    fig, ax = plt.subplots(3, figsize=[8, 8])
    for d in range(data.shape[1]):
        ax[0].plot(data[:,d])
    ax[1].pcolor(
        np.array(range(0, len(R[:,0]), sparsity)), 
        np.array(range(0, len(R[:,0]), sparsity)), 
        density_matrix, 
        cmap=cm.Greys, vmin=0, vmax=density_matrix.max(),
        shading='auto'
    )
    Nw=10
    out = R[Nw,Nw:-1]
    for i in range(50): # can't accept the first 50 elements
        out[i] = 0
    # ax[2].plot(R[Nw,Nw:-1])
    ax[2].plot(out)
    plt.legend(['Raw data with Original Changepoints'])
    # plt.show()
    plt.savefig(f"./fse-ss/{service_metric}_{case_idx}.png")
    plt.close()
    
    out = out.tolist()
    # write to file
    with open(f"./fse-ss/{service_metric}_{case_idx}.json", "w") as f:
        json.dump(out, f)
        
    anomalies = find_anomalies(out)[0]
    anomalies = list(map(int, anomalies))
    # write to file
    with open(f"./fse-ss/{service_metric}_{case_idx}_anomalies.json", "w") as f:
        json.dump(anomalies, f)
    # break 

100%|██████████| 100/100 [00:35<00:00,  2.84it/s]
